In [ ]:
def readGraph(fileName: str) -> [[int]]:
	file = open(fileName, "r")
	
	[n, m] = [int(x) for x in file.readline().split()]
	graph = [[] for i in range(n)]
	for i in range(m):
		[u, v] = [int(x) for x in file.readline().split()]
		graph[u].append(v)
		graph[v].append(u)
	
	file.close()
	
	return graph

def getParts(graph: [[int]]) -> ([int], [int]):
	n = len(graph)
	part = [0 for u in range(n)]
	
	def dfs(u: int) -> None:
		for v in graph[u]:
			if part[v] == 0:
				part[v] = part[u] ^ 3
				dfs(v)
			elif not part[u] ^ part[v] == 3:
				raise Exception("Graph is not bipartite.")
	
	for u in range(n):
		if part[u] == 0:
			part[u] = 1
			dfs(u)
	
	result = ([], [])
	for u in range(n):
		if part[u] == 1:
			result[0].append(u)
		else:
			assert part[u] == 2
			result[1].append(u)
	return result

In [ ]:
import queue

def HopcroftKarp(fileName: str) -> int:
	graph = readGraph(fileName)
	n = len(graph)
	(left, right) = getParts(graph)
	
	matched = [-1 for i in range(n)]
	
	while True:
		dist = [n + 1 for i in range(n)]
		def bfs() -> int:
			q = queue.Queue()
			
			for u in left:
				if matched[u] == -1:
					dist[u] = 0
					q.put(u)
			
			while not q.empty():
				u = q.get()
				for v in graph[u]:
					tmp = matched[v]
					if tmp == -1:
						return dist[u]
					elif dist[u] + 1 < dist[tmp]:
						dist[tmp] = dist[u] + 1
						q.put(tmp)
			
			return -1
		
		maxDist = bfs()
		if maxDist == -1:
			break
		
		def dfs(u: int) -> bool:
			for v in graph[u]:
				tmp = matched[v]
				if tmp == -1 or dist[tmp] == dist[u] + 1 and dfs(tmp):
					matched[u] = v
					matched[v] = u
					return True
			return False
		
		for u in left:
			if matched[u] == -1:
				dfs(u)
	
	result = 0
	for u in left:
		if matched[u] != -1:
			result += 1
	return result

In [ ]:
from mip import Model, xsum, maximize

def LP(fileName: str, solverName: str) -> int:
	graph = readGraph(fileName)
	n = len(graph)
	(left, right) = getParts(graph)
	
	m = 0
	edgesIds = [[] for u in range(n)]
	for u in left:
		for v in graph[u]:
			edgesIds[u].append(m)
			edgesIds[v].append(m)
			m += 1
	
	model = Model(solver_name=solverName)
	
	x = [model.add_var() for u in range(m)]
	for i in range(m):
		model += x[i] >= 0
	for u in range(n):
		model += xsum(x[i] for i in edgesIds[u]) <= 1
	
	model.objective = maximize(xsum(x))
	
	model.optimize()
	
	if model.objective_value == None:
		return 0
	return int(model.objective_value)

def LPcbc(fileName: str) -> int:
	return LP(fileName, "CBC")

def LPgur(fileName: str) -> int:
	return LP(fileName, "GRB")

In [ ]:
import random

def generateRandomGraph(n: int, p: float, fileName: str) -> None:
	random.seed(2137)
	
	edges = []
	for u in range(n):
		for v in range(n):
			if random.uniform(0.0, 1.0) <= p:
				edges.append((u, v + n))
	
	file = open(fileName, "w")
	file.write(f"{n * 2} {len(edges)}\n")
	for (u, v) in edges:
		file.write(f"{u} {v}\n")
	file.close()

In [ ]:
import time
import matplotlib.pyplot as plt

def compareMethods() -> None:
	n = 300
	
	xs = [i / 100 for i in range(101)]
	
	HopcroftKarpTimes = []
	LPcbcTimes = []
	LPgurTimes = []
	
	for x in xs:
		generateRandomGraph(n, x, "input")
		
		start_time = time.time()
		HopcroftKarpResult = HopcroftKarp("input")
		end_time = time.time()
		HopcroftKarpTimes.append(end_time - start_time)
		
		start_time = time.time()
		LPcbcResult = LPcbc("input")
		end_time = time.time()
		LPcbcTimes.append(end_time - start_time)
		
		start_time = time.time()
		LPgurResult = LPgur("input")
		end_time = time.time()
		LPgurTimes.append(end_time - start_time)
		
		# 300 even for p = 0.03 but whatever
		assert HopcroftKarpResult == LPcbcResult and LPcbcResult == LPgurResult
	
	plt.plot(xs, HopcroftKarpTimes, label='HopcroftKarp')
	plt.plot(xs, LPcbcTimes, label='LPcbc')
	plt.plot(xs, LPgurTimes, label='LPgur')
	plt.xlabel('Probability of edge existence')
	plt.ylabel('Time (seconds)')
	plt.title('Comparison of Methods')
	plt.legend()
	plt.show()

compareMethods()